<a href="https://colab.research.google.com/github/vrajeshtrichy/HuggingFace-GPT-RAG-MedicalQA/blob/master/HuggingFace-GPT-RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install elasticsearch sentence_transformers transformers eland==8.12.1 # accelerate # uncomment if using GPU
!pip install datasets==2.19.2 # Remove version lock if https://github.com/huggingface/datasets/pull/6978 has been released
!pip install openai

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from google.colab import userdata

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
CLOUD_ID = userdata.get("CLOUD_ID") # or "<YOUR CLOUD_ID>"

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = userdata.get("ELASTIC_API_KEY")  # or "<YOUR API KEY>"

In [22]:
from elasticsearch import Elasticsearch, helpers

# Create the client instance
client = Elasticsearch(cloud_id=CLOUD_ID, api_key=ELASTIC_API_KEY)

# Successful response!
client.info()

ObjectApiResponse({'name': 'instance-0000000000', 'cluster_name': 'a7730461ad0f483fbaff9fec75caa26e', 'cluster_uuid': '4i2TdQI3QJacLfbeUTQDtg', 'version': {'number': '8.15.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179', 'build_date': '2024-08-05T10:05:34.233336849Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [23]:
# Load Dataset
from datasets import load_dataset, Features, Value

# https://huggingface.co/datasets/MongoDB/whatscooking.restaurants

restaurant_dataset = load_dataset("MongoDB/whatscooking.restaurants")

print(restaurant_dataset)

DatasetDict({
    train: Dataset({
        features: ['restaurant_id', 'location', '_id', 'review_count', 'DogsAllowed', 'embedding', 'PriceRange', 'menu', 'HappyHour', 'TakeOut', 'address', 'sponsored', 'attributes', 'borough', 'OutdoorSeating', 'name', 'cuisine', 'stars'],
        num_rows: 25361
    })
})


In [29]:
# Data Preparation

# restaurant_dataset = restaurant_dataset.remove_columns("embedding")
# restaurant_dataset = restaurant_dataset.remove_columns("restaurant_id")
# restaurant_dataset = restaurant_dataset.remove_columns("location")
# restaurant_dataset = restaurant_dataset.remove_columns("_id")
# restaurant_dataset = restaurant_dataset.remove_columns("DogsAllowed")
# restaurant_dataset = restaurant_dataset.remove_columns("HappyHour")
# restaurant_dataset = restaurant_dataset.remove_columns("TakeOut")
# restaurant_dataset = restaurant_dataset.remove_columns("address")
# restaurant_dataset = restaurant_dataset.remove_columns("sponsored")
# restaurant_dataset = restaurant_dataset.remove_columns("OutdoorSeating")

# Remove data point where plot coloumn is missing

restaurant_dataset = restaurant_dataset.filter(lambda x: (
    (x["name"] is not None) and
     (x["cuisine"] is not None) and
      (x["borough"] is not None) and
       (x["menu"] is not None) and
        (x["review_count"] is not None) and
         (x["PriceRange"] is not None) and
          (x["attributes"] is not None) and
           (x["stars"] is not None)))

# Convert list of strings to strings
def list_to_string(example):
  for k, v in example.items():
    if isinstance(v, list):
      example[k] = ' '.join(v)
    elif isinstance(v, dict):
      example[k] = ' '.join(str(val) for val in v.values())
  return example

restaurant_dataset = restaurant_dataset.map(list_to_string)


restaurant_dataset = restaurant_dataset.cast(Features({
    "name": Value("string"),
    "cuisine": Value("string"),
    "borough": Value("string"),
    "menu": Value("string"),
    "review_count": Value("string"),
    "PriceRange": Value("string"),
    "attributes": Value("string"),
    "stars": Value("string")
    }))
# restaurant_dataset = restaurant_dataset.cast(Features({"review_count": Value("string")}))
# restaurant_dataset = restaurant_dataset.cast(Features({"PriceRange": Value("string")}))
# restaurant_dataset = restaurant_dataset.cast(Features({"attributes": Value("string")}))
# restaurant_dataset = restaurant_dataset.cast(Features({"stars": Value("string")}))

restaurant_dataset["train"]

Filter:   0%|          | 0/12775 [00:00<?, ? examples/s]

Map:   0%|          | 0/12775 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/12775 [00:00<?, ? examples/s]

Dataset({
    features: ['name', 'cuisine', 'borough', 'menu', 'review_count', 'PriceRange', 'attributes', 'stars'],
    num_rows: 12775
})

In [30]:
restaurant_dataset["train"][0]
# len(str(restaurant_dataset["train"][0]))

{'name': "Buddy'S Wonder Bar",
 'cuisine': 'American',
 'borough': 'Staten Island',
 'menu': 'Grilled cheese sandwich Baked potato Lasagna Mozzarella sticks Mac & cheese Chicken fingers Mashed potatoes Chicken pot pie Green salad Meatloaf Tomato soup Onion rings',
 'review_count': '62',
 'PriceRange': '2',
 'attributes': "'beer_and_wine' {'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True} None None True None None {'garage': False, 'street': True, 'validated': False, 'lot': True, 'valet': False} None None None True {'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False} True None u'average' 'casual' True True True True True u'free'",
 'stars': '3.5'}

In [31]:
USE_ELASTICSEARCH_VECTORISATION = True

EMBEDDING_MODEL_ID = "thenlper/gte-small"
# https://huggingface.co/thenlper/gte-small's page shows the dimensions of the model
# If you use the `gte-base` or `gte-large` embedding models, the numDimension
# value in the vector search index must be set to 768 and 1024, respectively.
EMBEDDING_DIMENSIONS = 384

In [32]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!(if [ "True" == $USE_ELASTICSEARCH_VECTORISATION ]; then \
  eland_import_hub_model --cloud-id $CLOUD_ID --hub-model-id $EMBEDDING_MODEL_ID --task-type text_embedding --es-api-key $ELASTIC_API_KEY --start --clear-previous; \
fi)

2024-08-27 13:46:27.981928: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-27 13:46:28.012664: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-27 13:46:28.019990: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-27 13:46:31.269757: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-08-27 13:46:34,136 INFO : Establishing connection to Elasticsearch
2024-08-27 13:46:34,667 INFO : Connected to cluster named 'a7730461ad0f483fbaff9fec75caa26e' (version: 8.15.0)
2024-08-27 13:46:34,667 INFO : Loading HuggingFace transfo

In [33]:
from sentence_transformers import SentenceTransformer

if not USE_ELASTICSEARCH_VECTORISATION:
    embedding_model = SentenceTransformer(EMBEDDING_MODEL_ID)


def get_embedding(text: str) -> list[float]:
    if USE_ELASTICSEARCH_VECTORISATION:
        raise Exception(
            f"Disabled when USE_ELASTICSEARCH_VECTORISATION is [{USE_ELASTICSEARCH_VECTORISATION}]"
        )
    else:
        if not text.strip():
            print("Attempted to get embedding for empty text.")
            return []

        embedding = embedding_model.encode(text)
        return embedding.tolist()


def add_fullplot_embedding(x):
    if USE_ELASTICSEARCH_VECTORISATION:
        raise Exception(
            f"Disabled when USE_ELASTICSEARCH_VECTORISATION is [{USE_ELASTICSEARCH_VECTORISATION}]"
        )
    else:
        full_details = x["name"] + x["borough"] + x["cuisine"] + x["stars"] + x["review_count"] + x["PriceRange"] + x["menu"] + x["attributes"]
        return {"embedding": [get_embedding(full_detail) for full_detail in full_details]}


if not USE_ELASTICSEARCH_VECTORISATION:
    restaurant_dataset = restaurant_dataset.map(add_fullplot_embedding, batched=True)
    restaurant_dataset["train"]

In [34]:
restaurant_dataset["train"]

Dataset({
    features: ['name', 'cuisine', 'borough', 'menu', 'review_count', 'PriceRange', 'attributes', 'stars'],
    num_rows: 12775
})

In [35]:
# restaurant_dataset["train"][0]

In [36]:
# Needs to match the id returned from Eland
# in general for Hugging Face models, you just replace the forward slash with
# double underscore
print(EMBEDDING_MODEL_ID)
model_id = EMBEDDING_MODEL_ID.replace("/", "__")

index_name = "restaurants"

index_mapping = {
    "properties": {
        "name": {"type": "text"},
        "borough": {"type": "text"},
        "cuisine": {"type": "text"},
        "stars": {"type": "text"},
        "review_count": {"type": "text"},
        "PriceRange": {"type": "text"},
        "menu": {"type": "text"},
        "attributes": {"type": "text"},
    }
}
# define index mapping
if USE_ELASTICSEARCH_VECTORISATION:
    index_mapping["properties"]["embedding"] = {
        "properties": {
            "is_truncated": {"type": "boolean"},
            "model_id": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "predicted_value": {
                "type": "dense_vector",
                "dims": EMBEDDING_DIMENSIONS,
                "index": True,
                "similarity": "cosine",
            },
        }
    }
else:
    index_mapping["properties"]["embedding"] = {
        "type": "dense_vector",
        "dims": EMBEDDING_DIMENSIONS,
        "index": "true",
        "similarity": "cosine",
    }

# flag to check if index has to be deleted before creating
should_delete_index = True

# check if we want to delete index before creating the index
if should_delete_index:
    if client.indices.exists(index=index_name):
        print("Deleting existing %s" % index_name)
        client.indices.delete(index=index_name, ignore=[400, 404])

print("Creating index %s" % index_name)


# ingest pipeline definition
if USE_ELASTICSEARCH_VECTORISATION:
    pipeline_id = "vectorize_restaurants"

    client.ingest.put_pipeline(
        id=pipeline_id,
        processors=[
            {
                "inference": {
                    "model_id": model_id,
                    "target_field": "embedding",
                    "field_map": {
                        "borough": "text_field",
                        "cuisine": "text_field",
                        "stars": "text_field",
                        "review_count": "text_field",
                        "PriceRange": "text_field",
                        "menu": "text_field",
                        "attributes": "text_field",
                        },
                }
            }
        ],
    )

    index_settings = {
        "index": {
            "default_pipeline": pipeline_id,
        }
    }
else:
    index_settings = {}

client.options(ignore_status=[400, 404]).indices.create(
    index=index_name, mappings=index_mapping, settings=index_settings
)

thenlper/gte-small
Deleting existing restaurants


<ipython-input-36-9c6e92b0ec5b>:53: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  client.indices.delete(index=index_name, ignore=[400, 404])


Creating index restaurants


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'restaurants'})

In [37]:
from elasticsearch.helpers import BulkIndexError

def batch_to_bulk_actions(batch):
    for record in batch:
        action = {
            "_index": "restaurants",
            "_source": {
                "name": record["name"],
                "borough": record["borough"],
                "cuisine": record["cuisine"],
                "stars": record["stars"],
                "review_count": record["review_count"],
                "price_range": record["PriceRange"],
                "menu": record["menu"],
                "attributes": record["attributes"],
            },
        }
        if not USE_ELASTICSEARCH_VECTORISATION:
            action["_source"]["embedding"] = record["embedding"]
        yield action


def bulk_index(ds):
    start = 0
    end = len(ds)
    batch_size = 100
    if USE_ELASTICSEARCH_VECTORISATION:
        # If using auto-embedding, bulk requests can take a lot longer,
        # so pass a longer request_timeout here (defaults to 10s), otherwise
        # we could get Connection timeouts
        batch_client = client.options(request_timeout=600)
    else:
        batch_client = client
    for batch_start in range(start, end, batch_size):
        batch_end = min(batch_start + batch_size, end)
        print(f"batch: start [{batch_start}], end [{batch_end}]")
        batch = ds.select(range(batch_start, batch_end))
        actions = batch_to_bulk_actions(batch)
        helpers.bulk(batch_client, actions)


try:
    bulk_index(restaurant_dataset["train"])
except BulkIndexError as e:
    print(f"{e.errors}")

print("Data ingestion into Elasticsearch complete!")

batch: start [0], end [100]
batch: start [100], end [200]
batch: start [200], end [300]
batch: start [300], end [400]
batch: start [400], end [500]
batch: start [500], end [600]
batch: start [600], end [700]
batch: start [700], end [800]
batch: start [800], end [900]
batch: start [900], end [1000]
batch: start [1000], end [1100]
batch: start [1100], end [1200]
batch: start [1200], end [1300]
batch: start [1300], end [1400]
batch: start [1400], end [1500]
batch: start [1500], end [1600]
batch: start [1600], end [1700]
batch: start [1700], end [1800]
batch: start [1800], end [1900]
batch: start [1900], end [2000]
batch: start [2000], end [2100]
batch: start [2100], end [2200]
batch: start [2200], end [2300]
batch: start [2300], end [2400]
batch: start [2400], end [2500]
batch: start [2500], end [2600]
batch: start [2600], end [2700]
batch: start [2700], end [2800]
batch: start [2800], end [2900]
batch: start [2900], end [3000]
batch: start [3000], end [3100]
batch: start [3100], end [320

In [39]:
def vector_search(restaurant_query):
    if USE_ELASTICSEARCH_VECTORISATION:
        knn = {
            "field": "embedding.predicted_value",
            "k": 10,
            "query_vector_builder": {
                "text_embedding": {
                    "model_id": model_id,
                    "model_text": restaurant_query,
                }
            },
            "num_candidates": 150,
        }
    else:
        question_embedding = get_embedding(restaurant_query)
        knn = {
            "field": "embedding",
            "query_vector": question_embedding,
            "k": 10,
            "num_candidates": 150,
        }

    response = client.search(index="restaurants", knn=knn, size=10)
    results = []
    for hit in response["hits"]["hits"]:
        id = hit["_id"]
        score = hit["_score"]
        name = hit["_source"]["name"]
        borough = hit["_source"]["borough"]
        cuisine = hit["_source"]["cuisine"]
        stars = hit["_source"]["stars"]
        review_count = hit["_source"]["review_count"]
        price_range = hit["_source"]["price_range"]
        menu = hit["_source"]["menu"]
        attributes = hit["_source"]["attributes"]
        result = {
            "id": id,
            "_score": score,
            "name": name,
            "borough": borough,
            "cuisine": cuisine,
            "stars": stars,
            "review_count": review_count,
            "price_range": price_range,
            "menu": menu,
            "attributes": attributes,
        }
        results.append(result)
    return results

def pretty_search(query):

    get_knowledge = vector_search(query)

    search_result = ""
    for result in get_knowledge:
        search_result += f"Name: {result.get('name', 'N/A')}, Location: {result.get('borough', 'N/A')}, Cuisine: {result.get('cuisine', 'N/A')}, Stars: {result.get('stars', 'N/A')}, Review_count: {result.get('review_count', 'N/A')}, PriceRange: {result.get('price_range', 'N/A')}, Menu: {result.get('menu', 'N/A')}, Attributes: {result.get('attributes', 'N/A')}\n"

    return search_result

In [40]:
# Conduct query with retrival of sources, combining results into something that
# we can feed to GPT
def combined_query(query):
    source_information = pretty_search(query)
    return f"Query: {query}\nContinue to answer the query by using these Search Results:\n{source_information}."


query = "What is the best restaurant for Kids in Manhattan?"
combined_results = combined_query(query)

print(combined_results)

Query: What is the best restaurant for Kids in Manhattan?
Continue to answer the query by using these Search Results:
Name: Keats Restaurant, Location: Manhattan, Cuisine: American, Stars: 4, Review_count: 149, PriceRange: 2, Menu: French fries Chicken pot pie Mac & cheese Chicken parmesan Lasagna Classic burger Chicken fingers Fried chicken Breadsticks Cheeseburger Mozzarella sticks Caesar salad, Attributes: None {'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': False} None None None None True {'garage': None, 'street': None, 'validated': None, 'lot': True, 'valet': False} None None None True {'dessert': None, 'latenight': None, 'lunch': True, 'dinner': None, 'brunch': None, 'breakfast': None} True None None u'casual' True True True None True u'free'
Name: Olive'S, Location: Manhattan, Cuisine: Bakery, Stars: 5, Review_count: 7, PriceRange: 1, Menu: doughnuts chocolate chip cookies 

In [50]:
from openai import OpenAI

openai_client = OpenAI(
  api_key=userdata.get("OPENAI_API_KEY")
)

# completion = openai_client.chat.completions.create(
#   model="gpt-4o-mini",
#   messages=[
#     {"role": "system", "content": "You are a highly intelligent restaurant recommending assistant, designed to assist users with restaurant search related queries. QUESTIONS AND ANSWERS SHOULD ONLY BE RELATED TO RESTAURANTS SEARCHING"},
#     {"role": "user", "content": "Hello!"}
#   ],
#   response_format={
#     "type": "text"
#   }
# )

# print(completion.choices[0].message.content)


In [51]:
messages = [
    {"role": "system", "content": "You are a highly intelligent restaurant recommending assistant, designed to assist users with restaurant search related queries. QUESTIONS AND ANSWERS SHOULD ONLY BE RELATED TO RESTAURANTS SEARCHING"},
    {"role": "user", "content": """Query: What is the best restaurant for Kids in Manhattan?
Continue to answer the query by using these Search Results:
Name: Keats Restaurant, Location: Manhattan, Cuisine: American, Stars: 4, Review_count: 149, PriceRange: 2, Menu: French fries Chicken pot pie Mac & cheese Chicken parmesan Lasagna Classic burger Chicken fingers Fried chicken Breadsticks Cheeseburger Mozzarella sticks Caesar salad, Attributes: None {'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': False} None None None None True {'garage': None, 'street': None, 'validated': None, 'lot': True, 'valet': False} None None None True {'dessert': None, 'latenight': None, 'lunch': True, 'dinner': None, 'brunch': None, 'breakfast': None} True None None u'casual' True True True None True u'free'
Name: Olive'S, Location: Manhattan, Cuisine: Bakery, Stars: 5, Review_count: 7, PriceRange: 1, Menu: doughnuts chocolate chip cookies chocolate pecan tart key lime pie, Attributes: None None None None None None True None None None None True None False None None 'casual' False True False False None None
Name: Palm Restaurant, Location: Manhattan, Cuisine: American, Stars: 3, Review_count: 8, PriceRange: 1, Menu: Chicken fingers Fried chicken Pigs in a blanket Cheddar Biscuits Mac & cheese Spaghetti with meatballs Chicken pot pie Baked potato Chicken parmesan Mushroom swiss burger Chicken soup Spinach cheese dip with chips, Attributes: u'none' {'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': False} None None None None True {'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False} False None None True {'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': False, 'breakfast': True} False None None u'casual' False True False None None u'no'
Name: Tap Room, Location: Manhattan, Cuisine: American, Stars: 4, Review_count: 21, PriceRange: 1, Menu: Mozzarella sticks Chicken parmesan Cheddar Biscuits Mashed potatoes Onion rings Green salad Pigs in a blanket Spaghetti with meatballs Chicken fingers Meatloaf Tomato soup Fried chicken, Attributes: None None None None True None True {'garage': False, 'street': None, 'validated': False, 'lot': None, 'valet': False} None None None None None None None None None None None None None None None
Name: Como Pizza, Location: Manhattan, Cuisine: Pizza, Stars: 4, Review_count: 60, PriceRange: 2, Menu: Pepperoni Pizza Diavola Hawaiian pizza brownies Greek salad Chef's Special Deluxe Pizza Garlic bread Desano cookies Margherita Pizza cheesy bread, Attributes: 'beer_and_wine' {'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True} None None True None None {'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False} False None None True {'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False} True None u'average' u'casual' False True True None None u'no'
Name: V & T Restaurant, Location: Manhattan, Cuisine: Italian, Stars: 1.5, Review_count: 16, PriceRange: 2, Menu: Minestrone soup Fried Mozzarella Pepperoni Pizza Manicotti White Pizza Vegetarian Broccoli Pizza All Meat Pizza Salmon chicken Buca Trio Platter caprese salad Alfredo Pizza, Attributes: None None None None False None True {'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False} False None None None None None None None None None None None None None 'free'
Name: Flame Restaurant Coffee House, Location: Manhattan, Cuisine: American, Stars: 4, Review_count: 366, PriceRange: 2, Menu: Mushroom swiss burger Spinach cheese dip with chips Classic burger Lasagna Grilled cheese sandwich Onion rings French fries Pigs in a blanket Meatloaf Spaghetti with meatballs Caesar salad Cheddar Biscuits, Attributes: 'none' {'touristy': False, 'hipster': None, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': None, 'upscale': False, 'classy': False, 'casual': True} None None True False True {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False} True None None True {'dessert': None, 'latenight': None, 'lunch': True, 'dinner': True, 'brunch': None, 'breakfast': False} False None u'average' 'casual' True True False False True u'free'
Name: Jackson Hole, Location: Manhattan, Cuisine: American, Stars: 3, Review_count: 10, PriceRange: 1, Menu: Chicken soup Onion rings Meatloaf Caesar salad Mushroom swiss burger Classic burger Green salad Grilled cheese sandwich Cheeseburger Mashed potatoes Breadsticks Spaghetti with meatballs, Attributes: None None None None True None True {'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False} None None None None None None None None None None None None None None 'no'
Name: John'S Restaurant, Location: Manhattan, Cuisine: Italian, Stars: 3, Review_count: 23, PriceRange: 2, Menu: Fried Mozzarella Neapolitan Pizza caprese salad Buca Trio Platter Alfredo with Spinach Pizza Alfredo Pizza Spaghetti Spinach Ravioli Cheese Ravioli chicken Vegetarian Baked Ziti White Pizza, Attributes: None None None None True False True {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False} False None None None None None None None None None None None None None None
Name: Corner Bistro, Location: Manhattan, Cuisine: American, Stars: 4, Review_count: 49, PriceRange: 1, Menu: Spinach cheese dip with chips Chicken soup Classic burger Cheddar Biscuits Lasagna Mozzarella sticks Mashed potatoes Meatloaf Baked potato Green salad Spaghetti with meatballs Fried chicken, Attributes: u'none' {'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': False} None None True None True {'garage': None, 'street': True, 'validated': False, 'lot': None, 'valet': False} False None None None None None None None None False None False False True u'free'"""},
    {"role": "assistant", "content": """Based on the provided search results, **Keats Restaurant** appears to be an excellent choice for kids in Manhattan. Here’s why:

1. **Kid-Friendly Menu**: Keats Restaurant offers a variety of dishes that are popular with kids, such as French fries, mac & cheese, chicken fingers, cheeseburgers, and mozzarella sticks. The menu is designed to appeal to young palates, making it a great spot for families.

2. **Casual Atmosphere**: The restaurant has a casual vibe, which is ideal for families with kids. Parents don’t have to worry about a formal or overly quiet environment, making it a comfortable setting for children.

3. **Parking Availability**: Keats Restaurant has a lot for parking, which is convenient for families who may be driving in. This can be especially important in Manhattan, where parking can often be a challenge.

4. **Lunch Availability**: Keats is open for lunch, which is a great option for families looking for a midday meal. The timing works well for families with young kids who may prefer to eat earlier in the day.

With its kid-friendly menu, casual setting, and convenient parking, Keats Restaurant stands out as a top option for a family-friendly dining experience in Manhattan."""},
]

In [52]:
def rag_query(query):

    combined_information = combined_query(query)

    messages.append({"role": "user", "content": combined_information})

    completion = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        response_format={
            "type": "text"
        }
        )

    messages.append({"role": "assistant", "content": completion.choices[0].message.content})

    return completion.choices[0].message.content



print(rag_query("What is the best restaurant for Kids in Brooklyn?"))

AttributeError: 'OpenAI' object has no attribute 'search'

In [46]:
search_query = input("Enter your search query: ")

print("\n\n########################################################## \n\n")

print("USER: ",search_query)

bot_response = rag_query(search_query)

print("XBOT: ",bot_response)

Enter your search query: Whats your name


########################################################## 


Your Query:  Whats your name
